# Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```
Or

```Python
!pip install crewai crewai_tools langchain_community
```

- Import from the crewAI libray.

In [3]:
!pip install crewai crewai_tools langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.0/192.0 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.0/468.0 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [111]:
# Dependencies
import os
import sys
import json
import requests
from crewai import Agent, Task, Crew, LLM
from crewai_tools import ScrapeWebsiteTool
scrapewebsite_tool = ScrapeWebsiteTool()

# Warning control
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Groq API

GROQ_API_KEY = 'gsk_'
# Visit https://groq.com/ to get API key

os.environ['GROQ_API_KEY'] = GROQ_API_KEY

# Definimos el modelo utilizado
llm = LLM(
    model="groq/llama-3.3-70b-versatile",
    temperature=0.7,
    #timeout=120,           # Seconds to wait for response
    max_tokens=750,       # Maximum length of response
    #top_p=0.9,            # Nucleus sampling parameter
    #frequency_penalty=0.1, # Reduce repetition
    #presence_penalty=0.1,  # Encourage topic diversity
    #response_format={"type": "json"},  # For structured outputs
    #seed=42               # For reproducible results
)


## Creating Agents

- Define your Agents, and, at least, provide them a `role`, `goal` and `backstory`.

  - Role: Defines the agent’s function and expertise within the crew.

  - Goal: The individual objective that guides the agent’s decision-making.

  - Backstory: Provides context and personality to the agent, enriching interactions.

  - Optional: `llm`, `tools`, `memory`, etc. (see: https://docs.crewai.com/concepts/agents)

- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [150]:
planner = Agent(
    role="Planificadora de Contenido",
    goal="Planifique contenido breve, atractivo y veraz sobre {topic}",
    backstory="Está trabajando en la planificación de un artículo de blog "
              "sobre el tema: {topic}."
              "Recopila información que ayuda a la audiencia a "
              "aprender sobre temas novedosos y a tomar decisiones "
              "informadas."
              "Su trabajo es la base para que el redactor de contenido "
              "escriba un artículo sobre este tema.",
    #tools=[scrapewebsite_tool],
    llm=llm,
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [151]:
writer = Agent(
    role="Escritora de Contenido",
    goal="Escribe un artículo de opinión breve, agudo y veraz "
         "sobre el tema: {topic}",
    backstory="Estás trabajando en la redacción de un "
              "nuevo artículo de opinión sobre el tema:: {topic}. "
              "Basas tu redacción en el trabajo de la "
              "Planificadora de Contenido, que proporciona un esquema "
              "y un contexto relevante sobre el tema. "
              "Sigues los objetivos principales y la dirección "
              "del esquema, tal como lo proporciona la "
              "Planificadora de Contenido. "
              "También proporcionas perspectivas objetivas e imparciales "
              "y las respaldas con la información proporcionada "
              "por la Planificadora de Contenido. "
              "Reconoces en tu artículo de opinión cuando tus declaraciones "
              "son opiniones en lugar de declaraciones objetivas.",
    #tools=[scrapewebsite_tool],
    llm=llm,
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [152]:
editor = Agent(
    role="Editora",
    goal="Editar una publicación de blog determinada para alinearla "
         "con el estilo de escritura de la organización. ",
    backstory="Usted es una Editora que recibe una publicación de blog "
              "de la Redactora de Contenido. "
              "Su objetivo es revisar la publicación de blog "
              "para asegurarse de que siga las mejores "
              "prácticas periodísticas, proporcione puntos de "
              "vista equilibrados al brindar opiniones o "
              "afirmaciones y también evite temas u "
              "opiniones controvertidos cuando sea posible.",
    llm=llm,
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [144]:
plan = Task(
    description=(
        "1. Priorizar las últimas tendencias, actores clave "
            "y noticias destacables sobre {topic}.\n"
        "2. Identificar el público objetivo, teniendo en cuenta "
            "sus intereses y temas sensibles.\n"
        "3. Desarrollar un esquema de contenido detallado que incluya "
            "una introducción, puntos clave y un llamado a la acción.\n"
        "4. Incluya palabras clave de optimización de motores de búsqueda "
            "(SEO, por sus siglas en inglés) y datos o fuentes relevantes."
    ),
    expected_output="Un documento de Plan de Contenido completo con "
                    "un esquema, análisis de audiencia, palabras clave "
                    "de SEO y datos o fuentes relevantes.",
    agent=planner,
)

### Task: Write

In [145]:
write = Task(
    description=(
        "1. Utilice el Plan de Contenido para elaborar una "
            "publicación de blog atractiva sobre {topic}.\n"
        "2. Incorporar palabras clave SEO de forma natural.\n"
		    "3. Las secciones/subtítulos deben estar nombrados "
            "de manera apropiada y atractiva.\n"
        "4. Asegúrese de que la publicación esté estructurada "
            "con una introducción atractiva, un cuerpo solido y "
            "una conclusión resumida.\n"
        "5. Corria errores gramaticales y alineación con el tono "
            "y la voz indicada.\n"
    ),
    expected_output="Una publicación de blog bien escrita en "
                    "formato Markdown, lista para publicar, "
                    "cada sección debe tener 2 o 3 párrafos.",
    agent=writer,
)

### Task: Edit

In [146]:
edit = Task(
    description=("Revise la publicación del blog proporcionada para "
                 "detectar errores gramaticales y alinearla con el "
                 "objetivo."),
    expected_output="Una publicación de blog bien escrita en formato "
                    "Markdown, lista para publicar, "
                    "cada sección debe tener 2 o 3 párrafos.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=TRUE` allows you to see all the logs of the execution.

In [153]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=True
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the class.

In [154]:
#

result = crew.kickoff(inputs={"topic": "Inteligencia Artificial"})

- Display the results of your execution as markdown in the notebook.

In [155]:
from IPython.display import Markdown
Markdown(result.raw) #Markdown(result)

# Introducción a la Inteligencia Artificial
La Inteligencia Artificial (IA) es un campo en constante evolución que ha revolucionado la forma en que vivimos y trabajamos. Desde asistentes virtuales hasta vehículos autónomos, la IA está transformando industrias y mejorando la eficiencia en todos los ámbitos. En este artículo, exploraremos las últimas tendencias, actores clave y noticias destacables sobre Inteligencia Artificial, y proporcionaremos información valiosa para que los lectores puedan tomar decisiones informadas sobre cómo aprovechar esta tecnología emergente.

La IA se refiere a la capacidad de las máquinas para realizar tareas que normalmente requieren inteligencia humana, como el aprendizaje, la resolución de problemas y la toma de decisiones. La IA se basa en algoritmos y modelos matemáticos que permiten a las máquinas aprender de los datos y mejorar su rendimiento con el tiempo. La IA tiene muchas aplicaciones en diferentes industrias, incluyendo la salud, las finanzas, la educación y la manufactura.

La historia de la IA se remonta a la década de 1950, cuando los primeros investigadores en inteligencia artificial comenzaron a explorar la posibilidad de crear máquinas que pudieran pensar y aprender como los seres humanos. Desde entonces, la IA ha avanzado significativamente, con avances en áreas como el aprendizaje automático, el procesamiento de lenguaje natural y la visión artificial.

## Tendencias Actuales en Inteligencia Artificial
Las tendencias actuales en IA incluyen el aprendizaje automático y el deep learning, que permiten a las máquinas aprender de grandes conjuntos de datos y mejorar su rendimiento con el tiempo. El procesamiento de lenguaje natural y los chatbots también son áreas de gran interés, ya que permiten a las máquinas entender y responder a las preguntas y solicitudes de los seres humanos de manera más natural y eficiente.

La visión artificial y el reconocimiento de patrones también son áreas importantes de la IA, ya que permiten a las máquinas identificar y clasificar objetos y patrones en imágenes y videos. Estas tecnologías tienen muchas aplicaciones en áreas como la seguridad, la salud y la manufactura. La IA también se está utilizando en la educación, para crear sistemas de aprendizaje personalizados y adaptativos que se ajusten a las necesidades individuales de cada estudiante.

Además, la IA se está utilizando en la industria de la salud, para analizar grandes conjuntos de datos y identificar patrones y tendencias que puedan ayudar a los médicos a diagnosticar y tratar enfermedades de manera más efectiva. La IA también se está utilizando en la industria de las finanzas, para analizar riesgos y hacer predicciones sobre el mercado.

## Actores Clave en la Inteligencia Artificial
Los actores clave en la IA incluyen empresas líderes en la industria, como Google, Amazon, Microsoft y Facebook, que están invirtiendo grandes cantidades de dinero en la investigación y el desarrollo de tecnologías de IA. También hay muchos investigadores y expertos en IA que están trabajando en universidades y centros de investigación, como Andrew Ng, Yann LeCun y Fei-Fei Li, que están avanzando en el estado

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [ ]:
topic = "YOUR TOPIC HERE"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on YOUR TOPIC HERE.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan for Blog Article on "The Importance of Mental Health Awareness"

1. Latest Trends, Key Players, and Noteworthy News:
- Latest Trends: The growing awareness and acceptance of mental health issues in society, especially due to the impact of the COVID-19 pandemic.
- Key Players: Organizations like NAMI (National Alliance on Mental Illness) and Mental Health America, as well as celebrities and influencers who are advocating for mental health awareness.
- Noteworthy News: Research studies on the effects of s

I now can give a great answer

Final Answer:
# The Importance of Mental Health Awareness

In today's fast-paced and often stressful society, mental health awareness has become increasingly crucial. The impact of the COVID-19 pandemic has further highlighted the importance of understanding and addressing mental health issues. Organizations like NAMI and Mental Health America, along with prominent celebrities and influencers, have been instrumental in advocating for mental health awareness and breaking the stigma surrounding mental illness.

Research studies have shed light on the effects of social media on mental health, leading to a greater emphasis on promoting mental health education and support. New therapies and treatments for mental health conditions are continuously being developed, offering hope to those struggling with mental health challenges.

For young adults aged 18-35, who are more susceptible to mental health issues due to various life stressors, the need for mental healt

In [ ]:
Markdown(result)

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).